In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/mind-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/mind-news-dataset


In [18]:
print(articles_df.columns)
print(user_histories_df.columns)
print(behaviors_df.columns)

Index(['id', 'category', 'subcategory', 'title', 'abstract', 'url', 'entity',
       'misc', 'abstract_clean', 'content'],
      dtype='object')
Index(['user_id', 'clicked_articles', 'history'], dtype='object')
Index(['impression_id', 'user_id', 'timestamp', 'history', 'impressions',
       'clicked_articles', 'all_articles'],
      dtype='object')


In [35]:
pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [1]:
pip install faiss-cpu sentence-transformers pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [3]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from bs4 import BeautifulSoup # For cleaning text
import re # For cleaning text

# --- Configuration for Paths ---
# Path to the MIND dataset input from Kaggle
MIND_INPUT_DIR = "/kaggle/input/mind-news-dataset"

# Output directory for generated files (within /kaggle/working/ for Kaggle notebooks)
# This is where cleaned_articles.csv and recommendations will be saved.
WORKING_DIR = "/kaggle/working"
OUTPUT_DATA_DIR = os.path.join(WORKING_DIR, "data")
os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

NEWS_TSV_PATH = os.path.join(MIND_INPUT_DIR, 'MINDsmall_train', '/kaggle/input/mind-news-dataset/news.tsv/news.tsv')
BEHAVIORS_TSV_PATH = os.path.join(MIND_INPUT_DIR, 'MINDsmall_train', '/kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv')

# --- Helper function to clean text (used for 'abstract_clean') ---
def clean_text_simple(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# === Load News Data ===
def load_news_data(file_path):
    print(f"Attempting to load news data from: {file_path}")
    if not os.path.exists(file_path):
        print(f"ERROR: News file not found at {file_path}")
        return pd.DataFrame()
    try:
        # Correct column names based on your provided sample
        column_names = ['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'TitleEntities', 'AbstractEntities']
        df = pd.read_csv(file_path, sep='\t', header=None, names=column_names)
        df['NewsID'] = df['NewsID'].astype(str) # Ensure NewsID is string
        print(f"Successfully loaded {len(df)} rows from {file_path}.")
        # Create 'abstract_clean' from 'Abstract'
        df['abstract_clean'] = df['Abstract'].apply(clean_text_simple)
        return df.drop_duplicates(subset=['NewsID'])
    except Exception as e:
        print(f"Error loading or processing news data from {file_path}: {e}")
        return pd.DataFrame()

# === Load Behaviors Data ===
def load_behaviors_data(file_path):
    print(f"Attempting to load behaviors data from: {file_path}")
    if not os.path.exists(file_path):
        print(f"ERROR: Behaviors file not found at {file_path}")
        return pd.DataFrame()
    try:
        # Correct column names based on your provided sample
        column_names = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
        df = pd.read_csv(file_path, sep='\t', header=None, names=column_names)
        df['UserID'] = df['UserID'].astype(str) # Ensure UserID is string
        print(f"Successfully loaded {len(df)} behavior logs from {file_path}.")
        return df
    except Exception as e:
        print(f"Error loading or processing behaviors data from {file_path}: {e}")
        return pd.DataFrame()

# === Parse Impression Strings ===
def parse_clicked_articles_from_impressions(impression_str):
    clicked_news = []
    if pd.isna(impression_str) or not isinstance(impression_str, str):
        return clicked_news
    articles = impression_str.split(' ')
    for article_info in articles:
        if article_info.endswith('-1'):
            clicked_news.append(article_info[:-2]) # Remove '-1'
    return [str(nid) for nid in clicked_news] # Ensure NewsID is string

def parse_history_articles(history_str):
    if pd.isna(history_str) or not isinstance(history_str, str) or history_str.strip() == '':
        return []
    return [str(nid) for nid in history_str.strip().split(' ')] # Ensure NewsID is string

# === Generate BERT Embeddings ===
def compute_news_embeddings(news_df, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    if news_df.empty:
        print("News DataFrame is empty. Skipping embedding computation.")
        return np.array([]), news_df

    model = SentenceTransformer(model_name)
    
    news_df['Title'] = news_df['Title'].fillna('').astype(str)
    news_df['Abstract'] = news_df['Abstract'].fillna('').astype(str) # Use original Abstract for embedding
    news_df['content_for_embedding'] = news_df['Title'] + " " + news_df['Abstract']
    
    # Filter out rows with effectively empty content
    # We need to keep track of original indices to map embeddings back
    original_indices = news_df.index
    news_df_to_embed = news_df[news_df['content_for_embedding'].str.strip().str.len() > 0]
    
    if news_df_to_embed.empty:
        print("All news articles have empty content for embedding. Cannot compute embeddings.")
        # Return an empty array of appropriate shape if possible, or just empty
        # We also return the original news_df because it might have 'id' column needed later
        return np.array([]), news_df

    print(f"Computing embeddings for {len(news_df_to_embed)} articles with content...")
    content_list = news_df_to_embed['content_for_embedding'].tolist()
    embeddings_subset = model.encode(content_list, show_progress_bar=True, convert_to_numpy=True)
    
    # Create a full-size embedding matrix, initially with zeros
    # This ensures embeddings align with the full news_df
    num_total_articles = len(news_df)
    embedding_dim = embeddings_subset.shape[1] if embeddings_subset.size > 0 else 384 # Default dim

    full_embeddings = np.zeros((num_total_articles, embedding_dim))
    
    # Place computed embeddings into the correct rows using original indices of subset
    # Map news_df_to_embed.index back to their positional equivalent in the original news_df.index
    # This assumes news_df.index is a simple RangeIndex if it was reset, or matches original_indices
    # A safer way:
    if embeddings_subset.size > 0:
        for i, original_idx in enumerate(news_df_to_embed.index):
            # Find the position of original_idx in the news_df's current index
            loc = news_df.index.get_loc(original_idx)
            full_embeddings[loc] = embeddings_subset[i]
            
    return full_embeddings, news_df # Return news_df as it might have new 'content_for_embedding' column


# === Build User Profiles ===
def build_user_profiles(behaviors_df, news_to_idx_map, news_embeddings_matrix):
    user_profiles_map = {}
    if news_embeddings_matrix.size == 0:
        print("News embeddings are empty. Cannot build user profiles.")
        return user_profiles_map

    for user_id, group_data in tqdm(behaviors_df.groupby('UserID'), desc="Building user profiles"):
        all_interacted_article_ids = set()
        for _, row_data in group_data.iterrows():
            all_interacted_article_ids.update(parse_history_articles(row_data['History']))
            all_interacted_article_ids.update(parse_clicked_articles_from_impressions(row_data['Impressions']))
        
        valid_article_indices = []
        for nid in all_interacted_article_ids:
            if nid in news_to_idx_map and news_to_idx_map[nid] < news_embeddings_matrix.shape[0]:
                valid_article_indices.append(news_to_idx_map[nid])
        
        if valid_article_indices:
            # Select embeddings for these valid articles
            profile_article_embeddings = news_embeddings_matrix[valid_article_indices]
            # Filter out zero vectors (articles with no content or failed embedding)
            active_embeddings = profile_article_embeddings[np.abs(profile_article_embeddings).sum(axis=1) > 1e-5]
            if active_embeddings.shape[0] > 0:
                user_profiles_map[user_id] = np.mean(active_embeddings, axis=0)
    return user_profiles_map

# === Build FAISS Index ===
def build_faiss_index(news_embeddings_matrix):
    if news_embeddings_matrix.size == 0 or news_embeddings_matrix.shape[0] == 0:
        print("News embeddings are empty. Cannot build FAISS index.")
        return None, [] # Return None for index and empty list for map
    
    # Filter out zero-norm vectors (articles with no content/embedding) before adding to FAISS
    norms = np.linalg.norm(news_embeddings_matrix, axis=1)
    valid_indices_for_faiss = np.where(norms > 1e-5)[0]
    
    if len(valid_indices_for_faiss) == 0:
        print("All news embeddings are zero vectors. Cannot build FAISS index.")
        return None, []

    embeddings_for_faiss = news_embeddings_matrix[valid_indices_for_faiss].astype(np.float32) # FAISS expects float32
    
    dimension = embeddings_for_faiss.shape[1]
    index = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(embeddings_for_faiss) # Normalize for cosine similarity
    index.add(embeddings_for_faiss)
    
    return index, valid_indices_for_faiss # Return map of FAISS idx to original news_df idx

# === Get Popular News (Filtered) ===
def get_popular_news_filtered(behaviors_df, articles_with_details_set):
    click_counts = defaultdict(int)
    for _, row_data in behaviors_df.iterrows():
        for news_id in parse_clicked_articles_from_impressions(row_data['Impressions']):
            if news_id in articles_with_details_set: # Only count if we have details
                click_counts[news_id] += 1
    return [news_id for news_id, _ in sorted(click_counts.items(), key=lambda item: item[1], reverse=True)]

# === Recommend with FAISS (Filtered) ===
def recommend_faiss_filtered(user_profile_vector, faiss_index, faiss_to_news_df_idx_map,
                             all_news_df, articles_with_details_set,
                             already_clicked_ids_set, top_n=10, popular_news_list_filtered=[]):
    if faiss_index is None:
        return [nid for nid in popular_news_list_filtered if nid not in already_clicked_ids_set][:top_n]

    if user_profile_vector is None or np.linalg.norm(user_profile_vector) < 1e-5:
        return [nid for nid in popular_news_list_filtered if nid not in already_clicked_ids_set][:top_n]

    query_vector = user_profile_vector.reshape(1, -1).astype(np.float32)
    faiss.normalize_L2(query_vector)
    
    num_candidates_to_search = top_n * 10 # Increased candidates
    
    recommendations = []
    try:
        distances, faiss_indices = faiss_index.search(query_vector, num_candidates_to_search)
        
        for i in range(faiss_indices.shape[1]):
            faiss_idx = faiss_indices[0, i]
            if faiss_idx == -1: break # No more results

            original_news_df_idx = faiss_to_news_df_idx_map[faiss_idx]
            news_id_candidate = all_news_df.iloc[original_news_df_idx]['NewsID'] # Use 'NewsID'
            
            if news_id_candidate in articles_with_details_set and news_id_candidate not in already_clicked_ids_set:
                recommendations.append(news_id_candidate)
            if len(recommendations) == top_n:
                break
    except Exception as e:
        print(f"FAISS search error: {e}. Falling back to popularity.")
        # Fallback to popular if FAISS fails for any reason
        recommendations.extend([nid for nid in popular_news_list_filtered if nid not in already_clicked_ids_set and nid not in recommendations][:top_n - len(recommendations)])
        return recommendations[:top_n] # Ensure we don't exceed top_n

    # Supplement with popular if not enough recommendations from FAISS
    if len(recommendations) < top_n:
        needed_more = top_n - len(recommendations)
        for popular_nid in popular_news_list_filtered:
            if needed_more == 0: break
            if popular_nid not in recommendations and popular_nid not in already_clicked_ids_set:
                recommendations.append(popular_nid)
                needed_more -= 1
    return recommendations

# === Evaluation Metrics (remain the same) ===
def precision_at_k(actual, predicted, k):
    if not actual: return 0.0
    # Ensure predicted is a list/set
    if not isinstance(predicted, (list, set)): predicted = []
    # Ensure actual is a set for efficient lookup
    actual_set = set(actual)
    predicted_k = predicted[:k]
    return len(actual_set.intersection(predicted_k)) / min(k, len(predicted_k)) if predicted_k else 0.0

def recall_at_k(actual, predicted, k):
    if not actual: return 0.0
    if not isinstance(predicted, (list, set)): predicted = []
    actual_set = set(actual)
    predicted_k = predicted[:k]
    return len(actual_set.intersection(predicted_k)) / len(actual_set) if actual_set and predicted_k else 0.0

def ndcg_at_k(actual, predicted, k):
    if not actual: return 0.0
    if not isinstance(predicted, (list, set)): predicted = []
    actual_set = set(actual)
    predicted_k = predicted[:k]
    
    dcg = 0.0
    for i, p_item in enumerate(predicted_k):
        if p_item in actual_set:
            dcg += 1.0 / np.log2(i + 2) # rank is i+1, log base 2 of rank+1
            
    idcg = 0.0
    for i in range(min(len(actual_set), k)): # Ideal ranking
        idcg += 1.0 / np.log2(i + 2)
        
    return dcg / idcg if idcg > 0 else 0.0

# === Main Pipeline ===
def main():
    print("--- Starting Main Pipeline ---")
    
    # 1. Load Data
    print("STEP 1: Loading News and Behaviors Data...")
    news_df = load_news_data(NEWS_TSV_PATH)
    if news_df.empty:
        print("CRITICAL ERROR: News data is empty. Cannot proceed.")
        return
    
    behaviors_df = load_behaviors_data(BEHAVIORS_TSV_PATH)
    if behaviors_df.empty:
        print("CRITICAL ERROR: Behaviors data is empty. Cannot proceed.")
        return

    # Rename NewsID to 'id' for consistency with UI scripts, ensure abstract_clean exists
    news_df.rename(columns={'NewsID': 'id'}, inplace=True)
    if 'abstract_clean' not in news_df.columns:
        news_df['abstract_clean'] = news_df['Abstract'].apply(clean_text_simple)

    # Save this comprehensive news_df as 'cleaned_articles.csv'
    cleaned_articles_output_path = os.path.join(OUTPUT_DATA_DIR, "cleaned_articles.csv")
    news_df.to_csv(cleaned_articles_output_path, index=False)
    print(f"✅ Saved comprehensive news data ({len(news_df)} articles) to: {cleaned_articles_output_path}")

    articles_with_details_set = set(news_df['id'].astype(str).tolist())
    print(f"Total unique articles with details: {len(articles_with_details_set)}")

    news_to_idx_map = {news_id: i for i, news_id in enumerate(news_df['id'].astype(str).tolist())}

    # 2. Compute Embeddings
    print("\nSTEP 2: Computing News Embeddings...")
    news_embeddings_matrix, news_df_with_content_col = compute_news_embeddings(news_df.copy()) # Pass a copy
    if news_embeddings_matrix.size == 0:
        print("CRITICAL ERROR: Failed to compute news embeddings. Cannot proceed.")
        return
    # news_df is now updated if compute_news_embeddings modified it (e.g. added content_for_embedding)
    # For consistency, ensure the news_df used henceforth is the one returned
    news_df = news_df_with_content_col


    # 3. Build FAISS Index
    print("\nSTEP 3: Building FAISS Index...")
    faiss_index, faiss_to_news_df_idx_map = build_faiss_index(news_embeddings_matrix)
    if faiss_index is None:
        print("WARNING: FAISS index could not be built. Recommendations will rely on popularity.")

    # 4. Build User Profiles
    print("\nSTEP 4: Building User Profiles...")
    user_profiles_map = build_user_profiles(behaviors_df, news_to_idx_map, news_embeddings_matrix)
    print(f"Built profiles for {len(user_profiles_map)} users.")

    # 5. Get Popular News (Filtered)
    print("\nSTEP 5: Computing Filtered Popular News Baseline...")
    popular_news_list_filtered = get_popular_news_filtered(behaviors_df, articles_with_details_set)
    print(f"Generated {len(popular_news_list_filtered)} popular articles (that have details).")

    # 6. Generate Recommendations and Evaluate
    print("\nSTEP 6: Generating Recommendations and Evaluating...")
    TOP_K = 10
    all_precisions, all_recalls, all_ndcgs = [], [], []
    recommendations_for_output_file = []

    # Group by UserID for processing each user's session(s)
    for user_id, user_group_data in tqdm(behaviors_df.groupby('UserID'), desc="Processing Users"):
        user_clicked_history_for_filtering = set() # Articles seen by user (history + impression clicks)
        actual_positive_clicks_for_eval = set()    # Articles clicked '1' in current impressions for this user

        for _, row in user_group_data.iterrows():
            # History is for filtering out already seen items
            user_clicked_history_for_filtering.update(parse_history_articles(row['History']))
            # Impression clicks contribute to both filtering and evaluation
            impression_clicks = parse_clicked_articles_from_impressions(row['Impressions'])
            user_clicked_history_for_filtering.update(impression_clicks)
            actual_positive_clicks_for_eval.update(impression_clicks) # Assuming these are the targets

        # Filter these sets by articles we actually have details for
        user_clicked_history_for_filtering_final = {nid for nid in user_clicked_history_for_filtering if nid in articles_with_details_set}
        actual_positive_clicks_for_eval_final = {nid for nid in actual_positive_clicks_for_eval if nid in articles_with_details_set}

        user_profile_vector = user_profiles_map.get(user_id, None)
        
        recommended_ids = recommend_faiss_filtered(
            user_profile_vector, faiss_index, faiss_to_news_df_idx_map,
            news_df, articles_with_details_set, # Pass the full news_df here
            user_clicked_history_for_filtering_final, TOP_K, popular_news_list_filtered
        )
        recommendations_for_output_file.append({'user_id': user_id, 'recommended_news_ids': recommended_ids})

        if actual_positive_clicks_for_eval_final: # Only evaluate if there are valid targets
            all_precisions.append(precision_at_k(actual_positive_clicks_for_eval_final, recommended_ids, TOP_K))
            all_recalls.append(recall_at_k(actual_positive_clicks_for_eval_final, recommended_ids, TOP_K))
            all_ndcgs.append(ndcg_at_k(actual_positive_clicks_for_eval_final, recommended_ids, TOP_K))

    if all_precisions:
        print(f"\n✅ Evaluation Results (k={TOP_K}):")
        print(f"  Precision@{TOP_K}: {np.mean(all_precisions):.4f}")
        print(f"  Recall@{TOP_K}:    {np.mean(all_recalls):.4f}")
        print(f"  nDCG@{TOP_K}:      {np.mean(all_ndcgs):.4f}")
    else:
        print("\n⚠ No valid data for evaluation (e.g., no users with positive clicks in impressions).")

    # 7. Save Recommendations
    print("\nSTEP 7: Saving Recommendations...")
    recs_output_file_path = os.path.join(OUTPUT_DATA_DIR, "bert_faiss_recommendations.csv")
    pd.DataFrame(recommendations_for_output_file).to_csv(recs_output_file_path, index=False)
    print(f"✅ Recommendations saved to {recs_output_file_path}")
    print("--- Main Pipeline Finished ---")

if __name__ == "__main__":
    main()

--- Starting Main Pipeline ---
STEP 1: Loading News and Behaviors Data...
Attempting to load news data from: /kaggle/input/mind-news-dataset/news.tsv/news.tsv
Successfully loaded 51282 rows from /kaggle/input/mind-news-dataset/news.tsv/news.tsv.


/tmp/ipykernel_31/2556159749.py:28: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()


Attempting to load behaviors data from: /kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv
Successfully loaded 156965 behavior logs from /kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv.
✅ Saved comprehensive news data (51282 articles) to: /kaggle/working/data/cleaned_articles.csv
Total unique articles with details: 51282

STEP 2: Computing News Embeddings...
Computing embeddings for 51282 articles with content...


Batches:   0%|          | 0/1603 [00:00<?, ?it/s]


STEP 3: Building FAISS Index...

STEP 4: Building User Profiles...


...

Processing Users:  74%|███████▍  | 37150/50000 [04:54<01:42, 125.43it/s]

FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search e

Processing Users:  74%|███████▍  | 37163/50000 [04:54<01:42, 125.32it/s]

FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search error: 'NewsID'. Falling back to popularity.
FAISS search e

Processing Users:  74%|███████▍  | 37189/50000 [04:54<01:46, 120.81it/s]

# **Evaluation**

In [11]:
import pandas as pd

# Path to your files
news_path = "/kaggle/input/mind-news-dataset/news.tsv/news.tsv"
behaviors_path = "/kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv"

# Print columns for news.tsv
print("Columns in news.tsv:")
try:
    news_df = pd.read_csv(news_path, sep='\t', nrows=1)
    print(list(news_df.columns))
except Exception as e:
    print(f"Error reading {news_path}: {e}")

print("\nColumns in behaviors.tsv:")
try:
    behaviors_df = pd.read_csv(behaviors_path, sep='\t', nrows=1)
    print(list(behaviors_df.columns))
except Exception as e:
    print(f"Error reading {behaviors_path}: {e}")


Columns in news.tsv:
['N55528', 'lifestyle', 'lifestyleroyals', 'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By', "Shop the notebooks, jackets, and more that the royals can't live without.", 'https://assets.msn.com/labs/mind/AAGH0ET.html', '[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]', '[]']

Columns in behaviors.tsv:
['1', 'U13740', '11/11/2019 9:05:58 AM', 'N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801', 'N55689-1 N35729-0']
